<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/IMT_ex_merge_join_gapminder_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Lab: Joins e outras transformações dos dados

Referências e Materiais úteis para este Lab:

- [Notas de Aula](https://colab.research.google.com/github/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/IMT_Pandas_I.ipynb)
- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html#user-guide)
- [Merge and Joins](https://pandas.pydata.org/docs/user_guide/merging.html)
---



In [ ]:
import numpy as np
import pandas as pd

path = 'https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/'

# **CASE: Gapminder**

Os dados dos exercícios a seguir tem como fonte o site [Gapmider](https://www.gapminder.org/). 

<img src="https://www.gapminder.org/wp-content/themes/gapminder2/images/gapminder-logo.svg" width=500, align="center"> 

In [ ]:
%%capture
!wget https://github.com/open-numbers/ddf--gapminder--co2_emission/raw/master/ddf--entities--country.csv
!wget https://github.com/open-numbers/ddf--gapminder--co2_emission/raw/master/ddf--datapoints--yearly_co2_emissions_1000_tonnes--by--country--year.csv
!wget https://github.com/open-numbers/ddf--bp--energy/raw/master/ddf--datapoints--coal_electricity_generation_twh--by--geo--year.csv


## Q0. Inspecione os dados.
Inspecione as bases de dados antes de prosseguir. Acesse e faça ao menos um `head()` de cada conjunto de dados.


In [ ]:
country_entities = pd.read_csv('ddf--entities--country.csv')
country_entities.head()

,country,name
0,abkh,Abkhazia
1,abw,Aruba
2,afg,Afghanistan
3,ago,Angola
4,aia,Anguilla


In [ ]:
co2_emissions = pd.read_csv('https://github.com/open-numbers/ddf--gapminder--co2_emission/raw/master/ddf--datapoints--yearly_co2_emissions_1000_tonnes--by--country--year.csv')
co2_emissions.head()

,country,year,yearly_co2_emissions_1000_tonnes
0,abw,1959,719.95990
1,abw,1960,619.06728
2,abw,1961,646.08113
3,abw,1962,709.52273
4,abw,1963,679.64378


In [ ]:
coal_eletricity = pd.read_csv('https://github.com/open-numbers/ddf--bp--energy/raw/master/ddf--datapoints--coal_electricity_generation_twh--by--geo--year.csv')
coal_eletricity.head()

,geo,year,coal_electricity_generation_twh
0,argentina,1985,0.607
1,argentina,1986,1.013
2,argentina,1987,1.016
3,argentina,1988,1.586
4,argentina,1989,0.865


## Q1. `merge()`

Crie um DataFrame `df` das emissões de CO2 incluindo o nome do país. 


In [ ]:
df = pd.merge(country_entities,co2_emissions) 
df.head()

,country,name,year,yearly_co2_emissions_1000_tonnes
0,abw,Aruba,1959,719.95990
1,abw,Aruba,1960,619.06728
2,abw,Aruba,1961,646.08113
3,abw,Aruba,1962,709.52273
4,abw,Aruba,1963,679.64378


(Moodle) Verifique da lista abaixo quais países não tem dados para o ano mais recente da base.

Dica: 
1. Crie um DataFrame `df_lista` com a lista de países
2. Crie um DataFrame `df_temp` fazendo um `left join` com `df_lista`
3. Separe como `df_temp[df_temp.isna().any(axis=1)]` os valores nulos incluídos de `df_lista` no `left join`

In [ ]:
lista = ['USSR', 'Hong Kong, China', 'Honduras', 'Antarctica', 'Croatia', 'Haiti', 'Hungary',
       'Indonesia', 'India', 'Ireland', 'Iran', 'Iraq', 'Iceland',
       'Israel', 'Italy', 'Jamaica', 'Jordan', 'Japan', 'Kazakhstan', 'Monaco']

In [ ]:
df_lista = pd.DataFrame({'name': lista})

df_temp = pd.merge(df_lista, df[ df.year == df.year.max() ], how='left')
df_temp = df_temp[df_temp.isna().any(axis=1)]

list(df_temp.name.unique())

['USSR', 'Antarctica', 'Monaco']

## Q2. `merge()` 2

Exclua os países identificados na questão anterior da lista. Empregue o DataFrame `df` para identificar os 3 países que mais efetuaram emissões no ano mais recente da base. Quais são esses países? 

In [ ]:
lista = [x for x in lista if x not in list(df_temp.name.unique())]
lista

['Hong Kong, China',
 'Honduras',
 'Croatia',
 'Haiti',
 'Hungary',
 'Indonesia',
 'India',
 'Ireland',
 'Iran',
 'Iraq',
 'Iceland',
 'Israel',
 'Italy',
 'Jamaica',
 'Jordan',
 'Japan',
 'Kazakhstan']

In [ ]:
df[ (df.name.isin(lista)) & (df.year == df.year.max()) ].nlargest(3,'yearly_co2_emissions_1000_tonnes')

,country,name,year,yearly_co2_emissions_1000_tonnes
8937,ind,India,2018,2.656274e+06
9873,jpn,Japan,2018,1.162933e+06
9161,irn,Iran,2018,7.210043e+05


## Q3. `join()`

Procure obter o mesmo resultado DataFrame anterior que você construiu com o método `merge()`, empregando agora o método `join()`.

Dica: Empregue `set_index()`. 

**Cuidados**

```
ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
```

O problema aqui é um equívoco de como o join funciona: quando você diz `df.join(df_to_join,on='key')` isso, não significa join on `df.key == df_to_join.key` como se poderia supor à primeira vista, mas join on `df.key == df_to_join.index`. Por isso o erro!

In [ ]:
try:
  display( co2_emissions.join(country_entities, on='country') )
except:
  print('error')

try:
  display( co2_emissions.join(country_entities) )
except:
  print('error')

try:
  display( country_entities.join(co2_emissions, on='country') )
except:
  print('error')

try:
  display( co2_emissions.join(country_entities.index, on='country') )
except:
  print('error')

try:
  display( co2_emissions.join(country_entities.set_index('country'), on='country') )
except:
  print('error')



error
error
error
error


,country,year,yearly_co2_emissions_1000_tonnes,name
0,abw,1959,719.95990,Aruba
1,abw,1960,619.06728,Aruba
2,abw,1961,646.08113,Aruba
3,abw,1962,709.52273,Aruba
4,abw,1963,679.64378,Aruba
...,...,...,...,...
20378,zwe,2014,11971.40630,Zimbabwe
20379,zwe,2015,12173.09130,Zimbabwe
20380,zwe,2016,10816.28203,Zimbabwe
20381,zwe,2017,12035.72697,Zimbabwe


(Moodle) Qual comando join resulta no conjunto de dados correto? 

## Q4. `merge(how='inner')` & `apply()`

Crie um DataFrame de energia produzida a partir do carvão incluindo o nome do país. Empregue um  `inner join`.

Dica: veja por que o merge direto não funciona.


In [ ]:
country_entities.name = country_entities.name.apply(lambda x: x.lower())

In [ ]:
coal_eletricity_country_names = pd.merge(country_entities,coal_eletricity,left_on='name',right_on='geo')
coal_eletricity_country_names.head()

,country,name,geo,year,coal_electricity_generation_twh
0,arg,argentina,argentina,1985,0.607
1,arg,argentina,argentina,1986,1.013
2,arg,argentina,argentina,1987,1.016
3,arg,argentina,argentina,1988,1.586
4,arg,argentina,argentina,1989,0.865


(Moodle) Quantos países há na base produzida e a soma de todos os valores `coal_electricity_generation_twh`? 

In [ ]:
len( coal_eletricity_country_names.name.unique() ) 

23

In [ ]:
coal_eletricity_country_names['coal_electricity_generation_twh'].sum() 

137198.26802701

## Q5. `merge(how='right')` & `apply()`

Recrie o DataFrame anterior agora empregando `right (left) join` para incluir países que não foram encontrados em `country_entities`.



In [ ]:
coal_eletricity_country_names = pd.merge(country_entities,coal_eletricity, how='right',left_on='name',right_on='geo')
coal_eletricity_country_names.head()


,country,name,geo,year,coal_electricity_generation_twh
0,arg,argentina,argentina,1985,0.607
1,arg,argentina,argentina,1986,1.013
2,arg,argentina,argentina,1987,1.016
3,arg,argentina,argentina,1988,1.586
4,arg,argentina,argentina,1989,0.865


(Moodle) Quais dos países ou geo posições apresentam `name` `null` no DataFrame produzido? 

Dica: empregue `isnull()`

In [ ]:
coal_eletricity_country_names[ coal_eletricity_country_names.name.isnull() ].geo.unique()

array(['other_africa', 'other_asia_pacific', 'other_cis', 'other_europe',
       'other_middle_east', 'other_s_cent_america', 'russian_federation',
       'saudi_arabia', 'south_africa', 'south_korea', 'total_africa',
       'total_asia_pacific', 'total_cis', 'total_europe',
       'total_middle_east', 'total_north_america', 'total_s_cent_america',
       'total_world', 'united_arab_emirates', 'united_kingdom', 'us'],
      dtype=object)